In [ ]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip

--2025-05-10 19:15:05--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 183.131.227.248
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|183.131.227.248|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 540689175 (516M) [application/zip]
Saving to: ‘train_sample.zip’

train_sample.zip    100%[===================>] 515.64M  3.43MB/s    in 1m 47s  

2025-05-10 19:16:53 (4.81 MB/s) - ‘train_sample.zip’ saved [540689175/540689175]



In [ ]:
!unzip -qq train_sample.zip
!\rm train_sample.zip

In [ ]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip

--2025-05-10 19:19:06--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 183.131.227.248
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|183.131.227.248|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1092637852 (1.0G) [application/zip]
Saving to: ‘test_a.zip’

test_a.zip          100%[===================>]   1.02G  5.46MB/s    in 4m 8s   

2025-05-10 19:23:15 (4.20 MB/s) - ‘test_a.zip’ saved [1092637852/1092637852]



In [ ]:
!unzip -qq test_a.zip
!\rm test_a.zip

In [ ]:
pip install -i https://mirrors.aliyun.com/pypi/simple pandas

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -i https://mirrors.aliyun.com/pypi/simple scikit-learn

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -i https://mirrors.aliyun.com/pypi/simple resampy

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install librosa -i https://pypi.tuna.tsinghua.edu.cn/simple --default-timeout=100
!pip install resampy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [1]:
# 必要的PyTorch库
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
import os
import glob
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# 特征提取以及数据集的建立

In [6]:

# 初始化特征和标签列表
feature = []
label = []

# 建立类别标签，不同类别对应不同的数字
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2, 'candied_fruits': 3, 'carrots': 4, 'chips': 5,
              'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream': 11,
              'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon': 17,
              'soup': 18, 'wings': 19}

# 创建一个反向字典，用于从数字映射回类别名称
label_dict_inv = {v: k for k, v in label_dict.items()}

# 定义特征提取函数
def extract_features_with_noise(parent_dir, sub_dirs, max_file=10, file_ext="*.wav", noise_factor=0.005):
    """
    从音频文件中提取特征和标签，并添加随机噪声进行数据增强。

    参数:
    - parent_dir: 数据集的父目录
    - sub_dirs: 子目录列表，每个子目录对应一个类别
    - max_file: 每个类别最多处理的文件数量
    - file_ext: 文件扩展名，默认为 "*.wav"
    - noise_factor: 随机噪声的强度因子

    返回:
    - feature: 提取的特征列表（包含倒谱MFCC特征）
    - label: 对应的标签列表
    """
    label, feature = [], []  # 初始化标签和特征列表

    # 遍历每个子目录
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]):
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            
            # 加载音频文件
            X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
            
            # 添加随机噪声
            noise = np.random.randn(len(X))
            X = X + noise_factor * noise
            X = np.clip(X, -1.0, 1.0)  # 确保音频数据在 [-1, 1] 范围内
            
            # 提取倒谱MFCC特征
            mfcc_features = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            
            # 添加到特征列表
            feature.extend([mfcc_features])
    
    return [feature, label]

In [7]:
# 加载训练数据
# 定义训练数据的父目录
parent_dir = './train_sample/'

# 定义保存目录（当前未使用）
save_dir = "./"

# 定义类别名称数组，每个类别对应一个子目录
folds = sub_dirs = np.array(['aloe', 'burger', 'cabbage', 'candied_fruits',
                             'carrots', 'chips', 'chocolate', 'drinks', 'fries',
                             'grapes', 'gummies', 'ice-cream', 'jelly', 'noodles', 'pickles',
                             'pizza', 'ribs', 'salmon', 'soup', 'wings'])

# 调用新的特征提取函数，从音频文件中提取倒谱MFCC特征和标签
X, Y = extract_features_with_noise(parent_dir, sub_dirs, max_file=100, noise_factor=0.005)


100%|██████████| 35/35 [00:01<00:00, 29.41it/s]


In [8]:
# 数据分割，将数据集划分为训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

# 转换为NumPy数组，确保数据格式一致
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# 确保特征数据大小适配模型输入
# 计算每个样本的总特征数，并调整形状为 (N, 1, 高度, 宽度)
feature_size = X_train.shape[1]  # 获取特征维度
height, width = 40, feature_size // 40  # 假设倒谱MFCC特征的高度为40，宽度自动计算
if feature_size % 40 != 0:
    raise ValueError(f"特征维度 {feature_size} 无法整除高度 {height}，请检查特征提取逻辑。")

# 调整数据形状
X_train = X_train.reshape(-1, 1, height, width)
X_test = X_test.reshape(-1, 1, height, width)

# 转换为PyTorch张量，创建训练集和测试集的数据集对象
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(Y_train))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(Y_test))

In [9]:
# 数据加载器
# 定义批量大小
batch_size = 64

# 创建训练数据加载器，支持随机打乱数据
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 创建测试数据加载器，不打乱数据
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [10]:
# 定义音频分类器模型（ResNet）
class ResNetAudioClassifier(nn.Module):
    def __init__(self, num_classes=20):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        layers = []
        for _ in range(num_blocks):
            layers.append(nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [11]:
# 初始化模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 实例化 ResNet 模型
model = ResNetAudioClassifier(num_classes=20).to(device)

# 定义优化器，这里使用Adam优化器，自动调整学习率
optimizer = optim.Adam(model.parameters())

# 定义损失函数，这里使用交叉熵损失函数，适用于多分类任务
criterion = nn.CrossEntropyLoss()

In [12]:
# 打印模型结构
# 通过打印模型，可以查看模型的各层结构和参数信息
print(model)

ResNetAudioClassifier(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (layer2): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentu

In [13]:
def train_model(model, train_loader, test_loader, criterion, optimizer, 
                num_epochs=100, validate_every=10, show_progress_every=1):
    """
    训练模型并定期进行验证。

    参数:
    - model: 要训练的PyTorch模型。
    - train_loader: 训练数据集的DataLoader。
    - test_loader: 验证数据集的DataLoader。
    - criterion: 损失函数，用于计算模型预测与真实标签之间的误差。
    - optimizer: 优化算法，用于更新模型参数。
    - num_epochs: 训练的总轮数（默认值: 100）。
    - validate_every: 每隔多少轮进行一次验证（默认值: 10）。
    - show_progress_every: 每隔多少轮显示一次进度条（默认值: 1）。
    """
    for epoch in range(num_epochs):
        # 设置模型为训练模式，启用dropout和batch normalization等训练特性
        model.train()
        
        # 初始化训练损失、正确预测计数和样本总数
        train_loss = 0.0  # 累计训练损失
        train_correct = 0  # 累计正确预测的样本数
        train_total = 0  # 累计训练样本总数

        # 控制是否显示进度条
        loader = train_loader if (epoch+1) % show_progress_every == 0 else train_loader
        disable_progress = not ((epoch+1) % show_progress_every == 0)

        # 遍历训练数据集的每个批次
        for inputs, labels in tqdm(loader, 
                                 desc=f"Epoch {epoch+1}/{num_epochs}", 
                                 unit="batch",
                                 disable=disable_progress):
            # 将输入和标签移动到设备（CPU/GPU）
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 清零优化器的梯度，避免梯度累积
            optimizer.zero_grad()
            
            # 前向传播计算输出
            outputs = model(inputs)
            
            # 计算损失，衡量预测与真实标签的差距
            loss = criterion(outputs, labels)
            
            # 反向传播计算梯度
            loss.backward()
            
            # 使用优化器更新模型参数
            optimizer.step()
            
            # 累计训练损失
            train_loss += loss.item()
            
            # 获取预测结果并统计正确预测的数量
            _, predicted = torch.max(outputs.data, 1)  # 获取每个样本的预测类别
            train_total += labels.size(0)  # 累计样本总数
            train_correct += (predicted == labels).sum().item()  # 累计正确预测的样本数
        
        # 计算平均训练损失和准确率
        train_loss = train_loss / len(train_loader)  # 平均训练损失
        train_acc = 100 * train_correct / train_total  # 训练准确率（百分比）
        
        # 验证阶段（根据指定频率）
        if (epoch + 1) % validate_every == 0 or epoch == num_epochs - 1:
            # 设置模型为评估模式，禁用dropout等训练特性
            model.eval()
            
            # 初始化验证损失、正确预测计数和样本总数
            val_loss = 0.0  # 累计验证损失
            val_correct = 0  # 累计正确预测的样本数
            val_total = 0  # 累计验证样本总数
            
            # 禁用梯度计算以加速验证过程
            with torch.no_grad():
                # 遍历验证数据集的每个批次
                for inputs, labels in test_loader:
                    # 将输入和标签移动到设备（CPU/GPU）
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    # 前向传播计算输出
                    outputs = model(inputs)
                    
                    # 计算损失
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    
                    # 获取预测结果并统计正确预测的数量
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)  # 累计样本总数
                    val_correct += (predicted == labels).sum().item()  # 累计正确预测的样本数
            
            # 计算验证准确率
            val_acc = 100 * val_correct / val_total  # 验证准确率（百分比）
            print(f"\nEpoch {epoch+1}/{num_epochs} - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
        else:
            # 非验证轮次仅显示训练准确率
            if not disable_progress:
                print(f"\nEpoch {epoch+1}/{num_epochs} - Train Acc: {train_acc:.2f}%")
    
    # 训练完成后打印最终结果
    print("\nTraining completed!")
    print(f"Final Train Accuracy: {train_acc:.2f}%")
    
    # 确保最后一个轮次进行验证（如果未在最后一轮验证）
    # 如果最后一个轮次未进行验证，则在训练结束后进行一次最终验证
    if (num_epochs - 1) % validate_every != 0:
        # 设置模型为评估模式，禁用dropout等训练特性
        model.eval()
        
        # 初始化验证损失、正确预测计数和样本总数
        val_loss = 0.0  # 累计验证损失
        val_correct = 0  # 累计正确预测的样本数
        val_total = 0  # 累计验证样本总数
        
        # 禁用梯度计算以加速验证过程
        with torch.no_grad():
            # 遍历验证数据集的每个批次
            for inputs, labels in test_loader:
                # 将输入和标签移动到设备（CPU/GPU）
                inputs, labels = inputs.to(device), labels.to(device)
                
                # 前向传播计算输出
                outputs = model(inputs)
                
                # 计算损失
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # 获取预测结果并统计正确预测的数量
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)  # 累计样本总数
                val_correct += (predicted == labels).sum().item()  # 累计正确预测的样本数
        
        # 计算最终验证准确率
        val_acc = 100 * val_correct / val_total  # 验证准确率（百分比）
        print(f"Final Validation Accuracy: {val_acc:.2f}%")

In [14]:
# 训练模型
# 调用 train_model 函数，开始训练音频分类器模型
# 参数说明：
# - model: 要训练的模型实例
# - train_loader: 训练数据的 DataLoader
# - test_loader: 测试数据的 DataLoader，用于验证模型性能
# - criterion: 损失函数，用于计算预测误差
# - optimizer: 优化器，用于更新模型参数
# - num_epochs: 训练的总轮数，这里设置为 500
# - validate_every: 每隔 50 个 epoch 进行一次验证
# - show_progress_every: 每隔 50 个 epoch 显示一次训练进度
train_model(model, train_loader, test_loader, criterion, optimizer, 
           num_epochs=500, validate_every=50, show_progress_every=50)


Epoch 50/500: 100%|██████████| 13/13 [00:00<00:00, 324.61batch/s]




Epoch 50/500 - Train Acc: 96.62%, Val Acc: 54.00%


Epoch 100/500: 100%|██████████| 13/13 [00:00<00:00, 318.70batch/s]




Epoch 100/500 - Train Acc: 97.12%, Val Acc: 64.00%


Epoch 150/500: 100%|██████████| 13/13 [00:00<00:00, 329.70batch/s]



...


Epoch 400/500 - Train Acc: 99.62%, Val Acc: 68.00%


Epoch 450/500: 100%|██████████| 13/13 [00:00<00:00, 320.04batch/s]




Epoch 450/500 - Train Acc: 100.00%, Val Acc: 76.00%


Epoch 500/500: 100%|██████████| 13/13 [00:00<00:00, 325.94batch/s]


Epoch 500/500 - Train Acc: 100.00%, Val Acc: 72.00%

Training completed!
Final Train Accuracy: 100.00%
Final Validation Accuracy: 72.00%


In [15]:
def extract_test_features(test_dir, max_file=None, file_ext="*.wav"):
    """
    从测试音频文件中提取特征。

    参数:
    - test_dir: 测试数据的目录路径。
    - max_file: 最多处理的文件数量（默认值: None，表示处理所有文件）。
    - file_ext: 文件扩展名，默认为 "*.wav"。

    返回:
    - features: 提取的特征，作为 PyTorch FloatTensor。
    - file_list: 处理的文件路径列表。
    """
    features = []  # 初始化特征列表
    file_list = glob.glob(os.path.join(test_dir, file_ext))  # 获取指定目录下的所有音频文件
    if max_file is not None:  # 如果指定了最大文件数量
        file_list = file_list[:max_file]  # 截取前 max_file 个文件
    
    # 遍历文件列表，提取每个文件的特征
    for fn in tqdm(file_list):
        # 加载音频文件
        X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
        # 计算梅尔频谱（mel spectrogram），并取其均值作为特征
        mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        features.append(mels)  # 将特征添加到特征列表中
    
    features = np.array(features)  # 转换为 NumPy 数组
    # 返回特征和文件列表
    return torch.FloatTensor(features), file_list

def predict(model, test_features):
    """
    使用模型对测试特征进行预测。

    参数:
    - model: 训练好的 PyTorch 模型。
    - test_features: 测试数据的特征，作为 PyTorch 张量。

    返回:
    - predictions: 预测的类别名称列表。
    """
    model.eval()  # 设置模型为评估模式，禁用 dropout 等训练特性
    with torch.no_grad():  # 禁用梯度计算以加速预测过程
        test_features = test_features.to(device)  # 将特征移动到设备（CPU/GPU）
        outputs = model(test_features)  # 前向传播计算输出
        _, predicted = torch.max(outputs.data, 1)  # 获取每个样本的预测类别
        # 将预测的类别数字映射回类别名称
        return [label_dict_inv[p.item()] for p in predicted]



In [16]:
test_features, file_list = extract_test_features('./test_a/')
# 调整测试特征的形状以适配模型输入
# 这里将特征调整为 (N, 1, 16, 8) 的形状，其中 N 是样本数量

100%|██████████| 2000/2000 [01:13<00:00, 27.35it/s]


In [17]:
# 调整测试特征的形状为 (N, 1, 16, 8)，其中 N 是样本数量
# 1 表示单通道（灰度图像），16 和 8 是特征的高和宽
# 这种形状适配卷积神经网络的输入格式
test_features = test_features.view(-1, 1, 16, 8)


In [18]:
# 使用训练好的模型对测试特征进行预测
# 参数:
# - model: 训练好的音频分类器模型
# - test_features: 提取并调整形状后的测试数据特征
# 返回:
# - predictions: 测试数据的预测类别名称列表
predictions = predict(model, test_features)


In [22]:
# 将预测结果赋值给变量 preds
preds = predictions

# 获取测试文件的路径列表
path = glob.glob('./test_a/*.wav')

# 创建一个 DataFrame，将文件路径和预测标签对应起来
result = pd.DataFrame({'name': path, 'label': preds})

# 提取文件名（去掉路径部分），并更新到 'name' 列
result['name'] = result['name'].apply(lambda x: x.split('/')[-1])

# 将结果保存为 CSV 文件，文件名为 'submit.csv'，不包含索引列
result.to_csv('submit.csv', index=None)

In [23]:
# 统计测试数据目录中以 .wav 为扩展名的音频文件数量
!ls ./test_a/*.wav | wc -l

2000


In [24]:
# 使用 wc -l 命令统计 submit.csv 文件中的行数
# 这可以用来检查提交文件中包含的记录数量
!wc -l submit.csv

2001 submit.csv
